In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
#import to dataframe
df = pd.read_csv("../input/pokemon.csv")

In [ ]:
#let's see what we have here
df.columns

In [ ]:
# a quick standard check 
df.describe()

## Key Questions

For choosing my Pokemon,  I would like to know:
- How attack, defense, hp and speed variate between different generations?
- Is there a favorite type per generation?
- What is the most common ability?
- Is there a correlation between types and attack, defense, hp or speed?
- If you could only choose one pokemon to take into battle, what would be the best option?

In [ ]:
df.head()

### Question 1 - How attack, defense, hp and speed variate between different generations?

In [ ]:
#Let's great a df to plot our answer

df1 = df.loc[:,('hp','generation')]
df2 = df.loc[:,('attack','generation')]
df3 = df.loc[:,('defense','generation')]
df4 = df.loc[:,('speed','generation')]

df1['attribute'] = 'hp'
df1.rename(columns={'hp': 'value'}, inplace=True)

df2['attribute'] = 'attack'
df2.rename(columns={'attack': 'value'}, inplace=True)

df3['attribute'] = 'defense'
df3.rename(columns={'defense': 'value'}, inplace=True)

df4['attribute'] = 'speed'
df4.rename(columns={'speed': 'value'}, inplace=True)

frames = [df1, df2, df3, df4]

result = pd.concat(frames)

In [ ]:
# plotting
sns.factorplot(kind='box',        # Boxplot
               y='value',         # Y-axis - values for boxplot
               x='attribute',     # X-axis - first factor
               hue='generation',  # Second factor denoted by color
               data=result,       # Dataframe 
               size=10,           # Figure size (x100px)      
               aspect=1.6,        # Width = size * aspect 
               legend_out=False)  # Make legend inside the plot
plt.title('Attribute variation between Pokemon generations')

 Question 1 - How attack, defense, hp and speed variate between different generations?

It seems like the creators are really good in being consistent when creating new Pokemons. Generation 1 and 4 seem a bit over powered in comparison to the rest.

### Question 2 - Is there a favorite type per generation?

In [ ]:
#Preparing df for plotting the answer

df5 = df.loc[:,('type1','generation', 'name')]
df5 = df5.groupby(['type1', 'generation'], as_index = False).count()
#df5 = df5.pivot(index='generation', columns='type1', values='name')

df5 = pd.pivot_table(df5, 
                       index='generation', 
                       columns='type1', 
                       values='name', 
                       aggfunc=np.sum,
                       fill_value = 0,
                       margins=True)

# this is a dataframe counting pokemons according to their type1 and generation

In [ ]:
# now we will create a % of the value in df5

df6 = df5.div(df5['All'], axis='index') 
df6.drop('All', axis=1, inplace = True)
df6.drop('All', inplace = True)
df6 = df6*100

df6 = df6.T

In [ ]:
# Draw a heatmap with the numeric values in each cell
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(df6, annot=True, linewidths=.5, ax=ax)

plt.title('Distribution of Pokemon type within each generation')

Question 2 - Is there a favorite type per generation?

For the first 3 generations, water type was really popular. Generation 5 was the better balanced so far. Normal type never gets out of style.

### Question 3 - What is the most common ability?

In [ ]:
# Pre preparation of data

ab_df = df[['type1', 'abilities']].copy()

t = ab_df['abilities'].astype(str).str.replace('[', '')
t = t.str.replace(']', '')
t = t.str.replace("'", '')

temp_abilities = t.str.split(',', expand=True)

temp_abilities['type'] = ab_df['type1']

temp_abilities

#stack columns
df_abilities_0 = temp_abilities[['type', 0]].copy()
df_abilities_1 = temp_abilities[['type', 1]].copy()
df_abilities_2 = temp_abilities[['type', 2]].copy()
df_abilities_3 = temp_abilities[['type', 3]].copy()
df_abilities_4 = temp_abilities[['type', 4]].copy()
df_abilities_5 = temp_abilities[['type', 5]].copy()

df_abilities_0[['type', 'ability']] = temp_abilities[['type', 0]]
df_abilities_1[['type', 'ability']] = temp_abilities[['type', 1]]
df_abilities_2[['type', 'ability']] = temp_abilities[['type', 2]]
df_abilities_3[['type', 'ability']] = temp_abilities[['type', 3]]
df_abilities_4[['type', 'ability']] = temp_abilities[['type', 4]]
df_abilities_5[['type', 'ability']] = temp_abilities[['type', 5]]

df_abilities = pd.concat([df_abilities_0, df_abilities_1, df_abilities_2,
                         df_abilities_3, df_abilities_4, df_abilities_5],
                        ignore_index=True)

df_abilities = df_abilities[['type', 'ability']]

df_abilities = df_abilities[df_abilities['ability'].notnull()]


# Overall abilities

over_ability = df_abilities.groupby(['ability'], 
                                    as_index=False).count().sort_values(by=['type'],
                                                                        ascending=False).reset_index()

print('From ' + str(over_ability['ability'].size) + ' abilities, the top 5 more common are (in order):'
     + str(over_ability['ability'][0:5].values))